In [1]:
import pymorphy2
import re
from collections import defaultdict


In [2]:
class Question_analizier:
    morph = pymorphy2.MorphAnalyzer ()
    questions = []
    all_words = defaultdict(lambda: 0)
    
    def get_set (self, question):
        words = re.findall(r'[^\s!,.?":;0-9]+', question)
        ret_set = set ()
        
        for word in words:
            self.all_words [word] += 1
            ret_set.add (self.morph.parse (word)[0].normal_form)
            
        return ret_set
    
    def add_question (self, question):
        self.questions.append (set())
        self.questions [-1] = self.get_set (question)
        
    def expend_question (self, question_id, another_question):
        set_to_add = self.get_set (another_question)
        self.questions [question_id] += set_to_add
        
    def get_closest_question (self, question):
        words = self.get_set (question)
        least_distance = float ("inf")
        least_id = -1
        
        for i in range (len(self.questions)):
            words_in_common = len (words & self.questions [i]) + 1 #not to devide by 0
            words_at_all = len (words) + len (self.questions [i]) 
            
            if words_at_all/(words_in_common*words_in_common) < least_distance:
                least_distance = words_at_all/(words_in_common*words_in_common)
                least_id = i
                
        return least_id, least_distance
    
    def get_popular_themes (self, number):
        return sorted(self.all_words.items(), key=lambda item:item[1])[-number:]


In [3]:
class Question_interpriter:
    edited_questions = Question_analizier ()
    questions = []
    answers = []
    
    def add (self, question, answer):
        self.edited_questions.add_question (question)
        self.questions.append (question)
        self.answers.append (answer)
        
    def get_answer (self, question):
        least_id, least_distance = self.edited_questions.get_closest_question (question)
        
        if least_distance > 0: #should add some value, not 0
            return self.answers [least_id], least_id
        else:
            return 'may be this will help you\n' + 'question:\n' + questions [least_id] + '\nanswer:\n' + answers [least_id]
            
    def expend_question (self, question_id, question):
        self.edited_questions.expend_question (question_id, question)

In [4]:
test = Question_interpriter ()
test.add ("не течет горячая вода", "надо подождать и она включится")
test.add ("как заработать готокоины", "усердно кодить и участвовать в различных конкурсках")
test.add ("я купил ужин на готокоины, его не было, а смена закончилась. Могу ли я вернуть готокоины", ":(")
test.add ("у меня болит голова", "сходи к доктору. Ее номер 8*********")
test.add ("мне надо написать еще вопросов", "и еще ответов")